## Circuit cutting with automatic cut finding using the Circuit Knitting Toolbox

### Import relevant modules

In [1]:
import numpy as np
from qiskit import QuantumCircuit

from circuit_knitting_toolbox.circuit_cutting import WireCutter

### Create a circuit to cut

In [2]:
qc = QuantumCircuit(5)
for i in range(5):
    qc.h(i)
qc.cx(0, 1)
for i in range(2, 5):
    qc.t(i)
qc.cx(0, 2)
qc.rx(np.pi / 2, 4)
qc.rx(np.pi / 2, 0)
qc.rx(np.pi / 2, 1)
qc.cx(2, 4)
qc.t(0)
qc.t(1)
qc.cx(2, 3)
qc.ry(np.pi / 2, 4)
for i in range(5):
    qc.h(i)

qc.draw()

┌───┐                ┌─────────┐   ┌───┐   ┌───┐
q_0: ┤ H ├──■───────■─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤┌─┴─┐     │     ├─────────┤   ├───┤   ├───┤
q_1: ┤ H ├┤ X ├─────┼─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤├───┤   ┌─┴─┐   └─────────┘   └───┘   ├───┤
q_2: ┤ H ├┤ T ├───┤ X ├────────■──────────■─────┤ H ├
     ├───┤├───┤   └───┘        │        ┌─┴─┐   ├───┤
q_3: ┤ H ├┤ T ├────────────────┼────────┤ X ├───┤ H ├
     ├───┤├───┤┌─────────┐   ┌─┴─┐   ┌──┴───┴──┐├───┤
q_4: ┤ H ├┤ T ├┤ Rx(π/2) ├───┤ X ├───┤ Ry(π/2) ├┤ H ├
     └───┘└───┘└─────────┘   └───┘   └─────────┘└───┘

### Load a WireCutter object with our QuantumCircuit and find a cutting scheme that meets our criteria

In [3]:
cutter = WireCutter(qc)
cuts = cutter.cut_automatic(
    max_subcircuit_width=3,
    max_subcircuit_cuts=10,
    max_subcircuit_size=12,
    max_cuts=10,
    num_subcircuits=[2],
)

Exporting as a LP file to let you check the model that will be solved :  inf <class 'float'>
Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               300
Tried aggregator 3 times.
MIP Presolve eliminated 23 rows and 5 columns.
MIP Presolve modified 4 coefficients.
Aggregator did 22 substitutions.
Reduced MIP has 47 rows, 24 columns, and 142 nonzeros.
Reduced MIP has 15 binaries, 9 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.36 ticks)
Found incumbent of value 1.000000 after 0.01 sec. (0.51 ticks)
Probing fixed 6 vars, tightened 8 bounds.
Probing changed sense of 12 constraints.
Probing time = 0.00 sec. (0.06 ticks)
Cover probing fixed 0 vars, tightened 24 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 47 rows and 24 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.02 ticks)

Root node processing (before b&c):
  Real time             =    0

### Define a constant shots function, evaluate our circuit, and verify the results

In [4]:
def num_shots_fn(circuit):
    return 1024


unordered_probabilities, smart_order = cutter.evaluate(
    cuts, mode="sv", num_shots_fn=num_shots_fn, mem_limit=24, num_threads=4
)
ordered_probabilities, metrics = cutter.verify(
    cuts, unordered_probabilities, smart_order
)
print(metrics)

{'nearest': {'chi2': 0, 'Mean Squared Error': 3.752148580465971e-34, 'Mean Absolute Percentage Error': 7.760053442355182e-14, 'Cross Entropy': 2.599681088367842, 'HOP': 0.9004283905932733}, 'naive': {'chi2': 0, 'Mean Squared Error': 2.793021597327788e-33, 'Mean Absolute Percentage Error': 7.625290711954506e-14, 'Cross Entropy': 2.5996810883678414, 'HOP': 0.900428390593274}}
